In [1]:
#%run mlflow_make_experiment.ipynb

import pandas as pd
import numpy as np
import catboost as ctb
import xgboost as xgb
from itertools  import combinations

df_train = pd.read_hdf("../input/train_churn_pred.h5") 
df_test = pd.read_hdf("../input/test_churn_pred.h5") 

print(df_train.shape, df_test.shape)

(34999, 171) (35000, 170)


In [2]:
def fe(df):
    metrics = {"_".join( x.split("_")[:-1] ) for x in df_train.columns if "mou_6" in x}

    for metric in metrics:
        metric_months = ["{}_{}".format(metric, x) for x in [8, 7, 6]]

        mean_by_month = df[metric_months].mean(axis=0).to_dict()
        df[f"mean_{metric}"] = df[metric_months].mean(axis=1)
        df[f"sum_{metric}"] = df[metric_months].sum(axis=1)
        df[f"min_{metric}"] = df[metric_months].min(axis=1)
        df[f"max_{metric}"] = df[metric_months].max(axis=1)



        for pair in combinations(metric_months, 2):
            df[f"{pair[0]} - {pair[1]}"] = df[pair[0]] - df[pair[1]] 
            df[f"{pair[0]} / {pair[1]}"] = df[pair[0]] / df[pair[1]] 
            

            df[f"mean_{pair[0]} - {pair[0]}"] = mean_by_month[pair[0]] - df[pair[0]]
            df[f"mean_{pair[1]} - {pair[0]}"] = mean_by_month[pair[1]] - df[pair[1]]
            
            
            df[f"mean_{pair[0]} / {pair[0]}"] = mean_by_month[pair[0]] / df[pair[0]]
            df[f"mean_{pair[1]} / {pair[0]}"] = mean_by_month[pair[1]] / df[pair[1]]
            

    df["date_of_last_rech_6"] = pd.to_datetime( df["date_of_last_rech_6"] )
    df["date_of_last_rech_7"] = pd.to_datetime( df["date_of_last_rech_7"] )
    df["date_of_last_rech_8"] = pd.to_datetime( df["date_of_last_rech_8"] )       
            
    df["date_of_last_rech_6_m"] = df["date_of_last_rech_6"].dt.month
    df["date_of_last_rech_7_m"] = df["date_of_last_rech_7"].dt.month
    df["date_of_last_rech_8_m"] = df["date_of_last_rech_8"].dt.month

    df["date_of_last_rech_6_d"] = df["date_of_last_rech_6"].dt.day
    df["date_of_last_rech_7_d"] = df["date_of_last_rech_7"].dt.day
    df["date_of_last_rech_8_d"] = df["date_of_last_rech_8"].dt.day
    
    df["date_of_last_rech_data_6"] = pd.to_datetime( df["date_of_last_rech_data_6"] )
    df["date_of_last_rech_data_7"] = pd.to_datetime( df["date_of_last_rech_data_7"] )
    df["date_of_last_rech_data_8"] = pd.to_datetime( df["date_of_last_rech_data_8"] )

    df["date_of_last_rech_data_6_m"] = df["date_of_last_rech_data_6"].dt.month
    df["date_of_last_rech_data_7_m"] = df["date_of_last_rech_data_7"].dt.month
    df["date_of_last_rech_data_8_m"] = df["date_of_last_rech_data_8"].dt.month

    df["date_of_last_rech_data_6_d"] = df["date_of_last_rech_data_6"].dt.day
    df["date_of_last_rech_data_7_d"] = df["date_of_last_rech_data_7"].dt.day
    df["date_of_last_rech_data_8_d"] = df["date_of_last_rech_data_8"].dt.day
    
    df["diff_date_of_last_rech_8-6"] = df["date_of_last_rech_8"] - df["date_of_last_rech_6"]
    df["diff_date_of_last_rech_8-7"] = df["date_of_last_rech_8"] - df["date_of_last_rech_7"]

    return df

In [3]:
df_all = pd.concat([df_train, df_test]).reset_index(drop=True)
df_train.shape, df_test.shape, df_all.shape

((34999, 171), (35000, 170), (69999, 171))

In [4]:
df_all_fe = fe(df_all.copy())

In [5]:
df_train = df_all_fe[ df_all_fe["churn_probability"].notnull() ]
df_test = df_all_fe[ df_all_fe["churn_probability"].isnull() ].copy()

In [6]:
feats_395 = ['mean_loc_og_t2c_mou_8 - loc_og_t2c_mou_8', 'mean_std_og_mou_8 - std_og_mou_8', 'mean_total_ic_mou_8 / total_ic_mou_8', 'roam_og_mou_8 / roam_og_mou_7', 'std_og_t2f_mou_6', 'mean_std_og_t2m_mou', 'max_rech_amt_8', 'spl_og_mou_8 / spl_og_mou_7', 'min_roam_og_mou', 'std_ic_t2m_mou_8 - std_ic_t2m_mou_7', 'loc_og_t2c_mou_8 - loc_og_t2c_mou_6', 'arpu_2g_7', 'mean_isd_og_mou', 'max_rech_data_7', 'loc_ic_mou_7 / loc_ic_mou_6', 'min_loc_ic_mou', 'sum_spl_og_mou', 'loc_ic_t2f_mou_8 / loc_ic_t2f_mou_6', 'max_rech_amt_7', 'mean_total_og_mou_7 - total_og_mou_7', 'sum_std_og_t2t_mou', 'mean_std_ic_t2t_mou_8 / std_ic_t2t_mou_8', 'count_rech_3g_7', 'last_day_rch_amt_7', 'total_ic_mou_8 - total_ic_mou_7', 'mean_std_og_mou_7 - std_og_mou_8', 'total_og_mou_8 / total_og_mou_6', 'mean_loc_ic_mou_7 / loc_ic_mou_8', 'sum_loc_og_t2m_mou', 'total_ic_mou_8 / total_ic_mou_6', 'mean_loc_ic_mou_6 / loc_ic_mou_7', 'max_std_ic_t2m_mou', 'mean_loc_og_t2f_mou_8 / loc_og_t2f_mou_8', 'arpu_3g_6', 'sum_total_og_mou', 'offnet_mou_8 - offnet_mou_7', 'mean_loc_og_t2t_mou_7 / loc_og_t2t_mou_8', 'sum_loc_ic_mou', 'mean_onnet_mou_7 - onnet_mou_7', 'min_roam_ic_mou', 'onnet_mou_6', 'total_og_mou_7 / total_og_mou_6', 'arpu_2g_6', 'std_og_t2t_mou_7', 'std_ic_t2m_mou_8 - std_ic_t2m_mou_6', 'max_total_og_mou', 'mean_total_ic_mou_7 / total_ic_mou_8', 'std_og_t2m_mou_8 - std_og_t2m_mou_6', 'spl_ic_mou_8', 'mean_loc_ic_t2m_mou_6 - loc_ic_t2m_mou_8', 'std_og_mou_7 - std_og_mou_6', 'mean_loc_ic_t2f_mou_7 / loc_ic_t2f_mou_7', 'spl_ic_mou_7 / spl_ic_mou_6', 'onnet_mou_8 - onnet_mou_6', 'loc_og_t2f_mou_8 / loc_og_t2f_mou_7', 'onnet_mou_7', 'jun_vbc_3g', 'std_ic_t2t_mou_7', 'mean_loc_ic_t2m_mou', 'mean_onnet_mou_6 - onnet_mou_8', 'mean_roam_ic_mou_8 / roam_ic_mou_8', 'mean_std_ic_t2t_mou_6 / std_ic_t2t_mou_7', 'mean_std_og_mou_6 / std_og_mou_7', 'min_onnet_mou', 'fb_user_6', 'mean_total_ic_mou_7 / total_ic_mou_7', 'loc_og_t2c_mou_7 - loc_og_t2c_mou_6', 'mean_loc_og_mou_7 - loc_og_mou_7', 'spl_ic_mou_8 - spl_ic_mou_7', 'mean_loc_og_t2t_mou_7 / loc_og_t2t_mou_7', 'offnet_mou_7', 'mean_total_og_mou_6 - total_og_mou_8', 'std_og_t2m_mou_8 / std_og_t2m_mou_6', 'mean_offnet_mou_6 - offnet_mou_7', 'std_og_t2m_mou_7 - std_og_t2m_mou_6', 'loc_og_t2t_mou_7 - loc_og_t2t_mou_6', 'mean_total_ic_mou_6 / total_ic_mou_7', 'loc_og_t2t_mou_6', 'onnet_mou_8 - onnet_mou_7', 'mean_loc_ic_t2m_mou_6 / loc_ic_t2m_mou_8', 'random_cats', 'mean_roam_ic_mou_7 / roam_ic_mou_7', 'std_og_t2f_mou_7 - std_og_t2f_mou_6', 'mean_std_og_mou', 'max_isd_og_mou', 'max_offnet_mou', 'sum_offnet_mou', 'arpu_8', 'loc_ic_t2m_mou_6', 'mean_onnet_mou_8 - onnet_mou_8', 'max_std_og_mou', 'roam_og_mou_6', 'mean_loc_ic_mou_7 / loc_ic_mou_7', 'max_loc_ic_t2t_mou', 'max_spl_og_mou', 'max_spl_ic_mou', 'total_ic_mou_8 / total_ic_mou_7', 'roam_ic_mou_8 - roam_ic_mou_6', 'last_day_rch_amt_8', 'offnet_mou_8 / offnet_mou_6', 'loc_ic_t2t_mou_8 / loc_ic_t2t_mou_6', 'mean_total_og_mou_8 - total_og_mou_8', 'total_rech_data_8', 'mean_total_ic_mou_8 - total_ic_mou_8', 'onnet_mou_8 / onnet_mou_7', 'isd_og_mou_8', 'mean_loc_og_t2c_mou_7 - loc_og_t2c_mou_7', 'std_ic_t2t_mou_8 - std_ic_t2t_mou_6', 'mean_roam_ic_mou_8 - roam_ic_mou_8', 'vol_2g_mb_7', 'max_rech_amt_6', 'min_loc_og_t2t_mou', 'mean_roam_ic_mou_7 - roam_ic_mou_8', 'loc_og_t2t_mou_8 - loc_og_t2t_mou_7', 'spl_og_mou_7', 'spl_ic_mou_6', 'sachet_2g_6', 'mean_roam_og_mou_7 / roam_og_mou_7', 'mean_loc_ic_t2t_mou_8 - loc_ic_t2t_mou_8', 'roam_ic_mou_8', 'mean_roam_og_mou', 'std_ic_mou_8 - std_ic_mou_6', 'loc_og_t2f_mou_8 - loc_og_t2f_mou_6', 'jul_vbc_3g', 'sum_roam_og_mou', 'max_loc_ic_t2f_mou', 'loc_ic_mou_8 - loc_ic_mou_6', 'mean_loc_ic_t2m_mou_7 / loc_ic_t2m_mou_8', 'mean_loc_og_t2m_mou_7 - loc_og_t2m_mou_8', 'offnet_mou_7 - offnet_mou_6', 'mean_std_ic_t2t_mou_6 - std_ic_t2t_mou_8', 'sum_loc_og_mou', 'mean_loc_ic_t2t_mou', 'onnet_mou_7 / onnet_mou_6', 'min_isd_ic_mou', 'loc_ic_t2t_mou_8 - loc_ic_t2t_mou_7', 'sum_onnet_mou', 'spl_ic_mou_7 - spl_ic_mou_6', 'std_og_t2o_mou', 'random_normal', 'mean_roam_og_mou_8 - roam_og_mou_8', 'max_onnet_mou', 'fb_user_7', 'mean_loc_ic_mou_8 - loc_ic_mou_8', 'roam_ic_mou_7 - roam_ic_mou_6', 'mean_std_og_t2m_mou_6 / std_og_t2m_mou_8', 'mean_loc_ic_t2m_mou_6 - loc_ic_t2m_mou_7', 'std_og_mou_8 - std_og_mou_6', 'mean_std_og_t2t_mou_7 - std_og_t2t_mou_7', 'mean_std_ic_t2m_mou_7 / std_ic_t2m_mou_8', 'mean_loc_og_t2t_mou_8 - loc_og_t2t_mou_8', 'mean_loc_og_t2c_mou_6 - loc_og_t2c_mou_7', 'std_og_mou_6', 'mean_loc_og_t2m_mou_8 - loc_og_t2m_mou_8', 'std_ic_t2m_mou_7 / std_ic_t2m_mou_6', 'mean_roam_ic_mou', 'loc_ic_t2t_mou_8 - loc_ic_t2t_mou_6', 'loc_ic_t2m_mou_7 / loc_ic_t2m_mou_6', 'mean_std_ic_t2t_mou_7 - std_ic_t2t_mou_8', 'vol_3g_mb_8', 'og_others_6', 'mean_roam_og_mou_6 / roam_og_mou_8', 'mean_roam_og_mou_8 / roam_og_mou_8', 'roam_og_mou_8', 'loc_og_t2c_mou_7 / loc_og_t2c_mou_6', 'mean_loc_og_mou_7 - loc_og_mou_8', 'spl_ic_mou_8 / spl_ic_mou_6', 'loc_og_t2o_mou', 'spl_og_mou_8 - spl_og_mou_7', 'mean_std_ic_t2t_mou_7 - std_ic_t2t_mou_7', 'mean_std_og_t2t_mou_6 - std_og_t2t_mou_7', 'loc_og_t2m_mou_8 - loc_og_t2m_mou_6', 'spl_og_mou_7 - spl_og_mou_6', 'fb_user_8', 'mean_std_og_mou_7 / std_og_mou_8', 'mean_loc_og_t2t_mou_6 / loc_og_t2t_mou_7', 'roam_ic_mou_8 / roam_ic_mou_6', 'loc_og_t2f_mou_7 - loc_og_t2f_mou_6', 'mean_isd_og_mou_8 / isd_og_mou_8', 'min_std_og_t2f_mou', 'mean_loc_ic_mou_6 - loc_ic_mou_7', 'onnet_mou_8', 'loc_og_mou_8 / loc_og_mou_7', 'total_rech_amt_8', 'mean_std_ic_t2f_mou', 'mean_std_og_t2t_mou_8 / std_og_t2t_mou_8', 'mean_spl_ic_mou_8 - spl_ic_mou_8', 'loc_ic_mou_8 - loc_ic_mou_7', 'total_ic_mou_7 / total_ic_mou_6', 'total_og_mou_8 / total_og_mou_7', 'mean_isd_ic_mou_6 - isd_ic_mou_7', 'mean_roam_og_mou_7 / roam_og_mou_8', 'mean_total_ic_mou_6 / total_ic_mou_8', 'std_ic_t2f_mou_8 / std_ic_t2f_mou_7', 'vol_3g_mb_7', 'min_isd_og_mou', 'total_rech_amt_6', 'mean_loc_ic_mou_7 - loc_ic_mou_8', 'total_og_mou_8 - total_og_mou_7', 'mean_loc_ic_t2m_mou_6 / loc_ic_t2m_mou_7', 'min_loc_og_t2c_mou', 'mean_loc_ic_t2t_mou_7 - loc_ic_t2t_mou_7', 'arpu_3g_8', 'sum_isd_og_mou', 'roam_og_mou_7 / roam_og_mou_6', 'mean_std_ic_t2m_mou_8 - std_ic_t2m_mou_8', 'loc_ic_t2t_mou_6', 'loc_ic_mou_6', 'max_loc_og_t2t_mou', 'min_std_og_t2m_mou', 'max_rech_data_6', 'loc_ic_t2f_mou_8 - loc_ic_t2f_mou_6', 'loc_ic_t2m_mou_8 - loc_ic_t2m_mou_6', 'loc_ic_mou_8', 'mean_isd_og_mou_7 / isd_og_mou_8', 'mean_loc_ic_mou_6 / loc_ic_mou_8', 'loc_ic_t2t_mou_8 / loc_ic_t2t_mou_7', 'onnet_mou_7 - onnet_mou_6', 'std_og_mou_8 / std_og_mou_7', 'mean_std_og_t2m_mou_8 / std_og_t2m_mou_8', 'mean_onnet_mou_8 / onnet_mou_8', 'total_rech_num_8', 'ic_others_6', 'mean_loc_ic_t2m_mou_8 / loc_ic_t2m_mou_8', 'loc_ic_t2f_mou_6', 'mean_loc_ic_t2f_mou_8 - loc_ic_t2f_mou_8', 'max_std_ic_mou', 'arpu_6', 'sum_isd_ic_mou', 'mean_offnet_mou_8 / offnet_mou_8', 'max_roam_og_mou', 'mean_onnet_mou_7 / onnet_mou_8', 'loc_ic_t2m_mou_8', 'loc_ic_mou_8 / loc_ic_mou_6', 'std_ic_t2t_mou_6', 'sum_std_ic_mou', 'roam_og_mou_8 - roam_og_mou_7', 'loc_og_t2m_mou_8 / loc_og_t2m_mou_6', 'mean_roam_og_mou_7 - roam_og_mou_8', 'mean_onnet_mou_6 - onnet_mou_7', 'loc_ic_t2m_mou_8 / loc_ic_t2m_mou_6', 'sum_roam_ic_mou', 'max_std_ic_t2t_mou', 'max_loc_ic_t2m_mou', 'ic_others_7', 'max_rech_data_8', 'aug_vbc_3g', 'loc_og_t2f_mou_8 - loc_og_t2f_mou_7', 'mean_offnet_mou_6 / offnet_mou_8', 'mean_spl_og_mou', 'loc_og_t2c_mou_8 - loc_og_t2c_mou_7', 'total_ic_mou_6', 'mean_spl_ic_mou_7 / spl_ic_mou_7', 'mean_loc_ic_t2f_mou_8 / loc_ic_t2f_mou_8', 'loc_og_t2m_mou_8', 'std_ic_t2m_mou_7 - std_ic_t2m_mou_6', 'arpu_3g_7', 'total_og_mou_8', 'mean_std_ic_mou_7 - std_ic_mou_7', 'min_offnet_mou', 'loc_ic_t2t_mou_7 / loc_ic_t2t_mou_6', 'std_og_t2f_mou_8 - std_og_t2f_mou_7', 'mean_spl_ic_mou', 'loc_ic_t2m_mou_7', 'std_og_mou_7', 'std_ic_mou_8 - std_ic_mou_7', 'std_og_t2t_mou_6', 'roam_og_mou_7', 'offnet_mou_8', 'last_day_rch_amt_6', 'min_loc_ic_t2t_mou', 'loc_ic_t2f_mou_7', 'mean_onnet_mou_6 / onnet_mou_8', 'isd_ic_mou_6', 'total_rech_num_7', 'mean_loc_ic_t2m_mou_7 / loc_ic_t2m_mou_7', 'sum_loc_ic_t2f_mou', 'mean_spl_og_mou_6 - spl_og_mou_7', 'sum_loc_og_t2t_mou', 'loc_ic_mou_7 - loc_ic_mou_6', 'max_loc_og_t2m_mou', 'mean_spl_og_mou_7 - spl_og_mou_7', 'av_rech_amt_data_8', 'std_ic_mou_6', 'mean_offnet_mou_7 - offnet_mou_7', 'max_loc_og_mou', 'night_pck_user_7', 'mean_std_ic_mou', 'total_ic_mou_7 - total_ic_mou_6', 'random_exponential', 'loc_ic_t2m_mou_8 / loc_ic_t2m_mou_7', 'mean_loc_ic_t2f_mou', 'mean_isd_ic_mou', 'mean_loc_og_t2f_mou_6 / loc_og_t2f_mou_8', 'mean_offnet_mou_8 - offnet_mou_8', 'mean_std_ic_t2t_mou_6 / std_ic_t2t_mou_8', 'sum_loc_ic_t2m_mou', 'total_og_mou_6', 'loc_ic_t2t_mou_7', 'mean_total_og_mou', 'max_loc_ic_mou', 'mean_loc_og_mou_6 - loc_og_mou_7', 'loc_ic_mou_8 / loc_ic_mou_7', 'roam_ic_mou_7 / roam_ic_mou_6', 'roam_ic_mou_7', 'std_og_t2t_mou_7 - std_og_t2t_mou_6', 'loc_og_mou_7 - loc_og_mou_6', 'mean_total_ic_mou', 'std_ic_t2t_mou_7 - std_ic_t2t_mou_6', 'mean_loc_og_mou_6 - loc_og_mou_8', 'loc_ic_t2f_mou_7 - loc_ic_t2f_mou_6', 'min_spl_og_mou', 'mean_std_ic_mou_6 - std_ic_mou_8', 'loc_og_t2t_mou_8', 'loc_ic_mou_7', 'loc_og_mou_7 / loc_og_mou_6', 'std_og_t2t_mou_8 - std_og_t2t_mou_6', 'count_rech_2g_6', 'std_og_t2m_mou_8', 'roam_og_mou_8 - roam_og_mou_6', 'isd_og_mou_8 - isd_og_mou_7', 'max_loc_og_t2f_mou', 'total_ic_mou_8 - total_ic_mou_6', 'mean_std_og_t2m_mou_6 - std_og_t2m_mou_7', 'mean_std_ic_t2m_mou_7 - std_ic_t2m_mou_8', 'total_ic_mou_8', 'loc_og_mou_7', 'mean_roam_ic_mou_7 - roam_ic_mou_7', 'std_ic_mou_7 / std_ic_mou_6', 'total_rech_num_6', 'total_rech_amt_7', 'mean_loc_og_t2c_mou_8 / loc_og_t2c_mou_8', 'mean_offnet_mou', 'mean_std_og_mou_6 - std_og_mou_8', 'spl_ic_mou_7', 'av_rech_amt_data_7', 'roam_ic_mou_8 / roam_ic_mou_7', 'mean_total_ic_mou_6 - total_ic_mou_7', 'night_pck_user_8', 'std_og_mou_8 - std_og_mou_7', 'mean_loc_og_t2c_mou', 'arpu_7', 'std_ic_t2m_mou_6', 'min_std_ic_t2m_mou', 'roam_ic_mou_8 - roam_ic_mou_7', 'mean_onnet_mou', 'max_total_ic_mou', 'mean_loc_ic_mou', 'std_og_t2m_mou_8 - std_og_t2m_mou_7', 'spl_ic_mou_8 / spl_ic_mou_7', 'loc_ic_t2m_mou_7 - loc_ic_t2m_mou_6', 'mean_loc_ic_t2m_mou_8 - loc_ic_t2m_mou_8', 'mean_loc_ic_t2f_mou_6 - loc_ic_t2f_mou_8', 'mean_loc_ic_mou_8 / loc_ic_mou_8', 'max_loc_og_t2c_mou', 'mean_loc_ic_t2f_mou_6 - loc_ic_t2f_mou_7', 'mean_loc_ic_t2t_mou_8 / loc_ic_t2t_mou_8', 'mean_loc_ic_t2t_mou_6 - loc_ic_t2t_mou_8', 'std_ic_t2m_mou_8 / std_ic_t2m_mou_7', 'loc_og_t2m_mou_8 - loc_og_t2m_mou_7', 'loc_og_t2m_mou_7 / loc_og_t2m_mou_6', 'roam_og_mou_8 / roam_og_mou_6', 'loc_ic_t2m_mou_8 - loc_ic_t2m_mou_7', 'vol_2g_mb_6', 'total_og_mou_8 - total_og_mou_6', 'mean_loc_og_t2t_mou_7 - loc_og_t2t_mou_8', 'aon', 'ic_others_8', 'std_og_t2t_mou_8 - std_og_t2t_mou_7', 'roam_og_mou_7 - roam_og_mou_6', 'mean_offnet_mou_6 - offnet_mou_8', 'mean_loc_og_t2m_mou', 'std_og_t2t_mou_7 / std_og_t2t_mou_6', 'loc_ic_t2t_mou_8', 'loc_ic_t2f_mou_8', 'mean_roam_ic_mou_7 / roam_ic_mou_8', 'loc_og_mou_8 - loc_og_mou_6', 'std_ic_t2f_mou_7 - std_ic_t2f_mou_6', 'vol_3g_mb_6', 'mean_loc_og_t2f_mou', 'sum_total_ic_mou', 'offnet_mou_8 / offnet_mou_7', 'mean_std_og_t2f_mou_8 - std_og_t2f_mou_8', 'mean_std_og_mou_8 / std_og_mou_8', 'mean_std_og_mou_6 - std_og_mou_7', 'onnet_mou_8 / onnet_mou_6', 'date_of_last_rech_data_8_d','date_of_last_rech_data_7_d', 'date_of_last_rech_data_6_d', 'date_of_last_rech_data_8_m','date_of_last_rech_data_7_m', 'date_of_last_rech_data_6_m', 'date_of_last_rech_8_d','date_of_last_rech_7_d', 'date_of_last_rech_6_d', 'date_of_last_rech_8_m','date_of_last_rech_7_m', 'date_of_last_rech_6_m']
feats_395 =  [x for x in feats_395 if "random" not in x]

In [7]:
X_train = df_train[feats_395].fillna(-1).values
y_train = df_train["churn_probability"].values

In [8]:
model = ctb.CatBoostClassifier(
    max_depth=7, n_estimators=200, random_state=0, 
    learning_rate=0.085, verbose=False
)

In [9]:
model.fit(X_train, y_train)
df_test["churn_probability"] = (model.predict_proba(df_test[feats_395].fillna(-1).values)[:,1] > 0.2).astype(np.int)

In [10]:
df_test[ ["id", "churn_probability"] ].to_csv("../output/ctb_thr_0.2_fe_feats_395.csv", index=False)

In [11]:
model_xgb = xgb.XGBClassifier(max_depth=7, n_estimators=200, random_state=0, learning_rate=0.085)

In [12]:
model_xgb.fit(X_train, y_train)
df_test["churn_probability"] = (model_xgb.predict_proba(df_test[feats_395].fillna(-1).values)[:,1] > 0.2).astype(np.int)

In [13]:
df_test[ ["id", "churn_probability"] ].to_csv("../output/xgb_thr_0.2_fe_feats_395.csv", index=False)

In [14]:
xgb395_output = pd.read_csv("../output/xgb_thr_0.2_fe_feats_395.csv")
ctb395_output = pd.read_csv("../output/ctb_thr_0.2_fe_feats_395.csv")

output = pd.DataFrame()
output["id"] = ctb395_output["id"]
output["churn_probability"] = ((0.60*ctb395_output["churn_probability"] + 0.4*xgb395_output["churn_probability"]) > 0.2).astype(np.int)

output.to_csv("../output/0.6ctb395_0.4xgb395.csv", index=False)